In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import anndata as ad
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (4, 4)
import pyarrow

In [2]:
# read in methylated gene body cytosine counts
countsdata = sc.read_h5ad('/u/project/jflint/heffel/BICAN3/210505/mcg_210505_raw_genebody_blacklist_counts.h5ad')

# read in gene body cytosine coverage
covdata = sc.read_h5ad('/u/project/jflint/heffel/BICAN3/210505/mcg_210505_raw_genebody_blacklist_coverage.h5ad')

In [3]:
covdata.shape

(5618, 53100)

In [4]:
countsdata.shape == covdata.shape

True

In [5]:
(countsdata.obs.index == covdata.obs.index).all()

np.True_

In [6]:
(countsdata.var.index == covdata.var.index).all()

np.True_

In [7]:
# create readable gene ID's (ie: FAM41C as opposed to ENSG00000230368.2)
conv_table = pd.read_csv("/u/project/cluo/heffel/BICAN/ensg_to_symbol.tsv", 
                         sep="\\t").set_index("ENSG").to_dict()["Symbol"] # read in csv file

# copy gene ID's to both AnnData objects
covdata.var.index = covdata.var.index.map(lambda x: conv_table.get(x, x))
countsdata.var.index = countsdata.var.index.map(lambda x: conv_table.get(x, x))

In [8]:
len(".allc.tsv.gz") # check length

12

In [9]:
# truncate (".allc.tsv.gz") from cells
countsdata.obs.index = [i[:-12] for i in countsdata.obs.index]
covdata.obs.index = [i[:-12] for i in covdata.obs.index]

In [10]:
countsdata.write_h5ad("mcg_210505_raw_genebody_blacklist_counts.h5ad")
covdata.write_h5ad("mcg_210505_raw_genebody_blacklist_coverage.h5ad")